이 파일은 item x item 상호작용 matrix를 cluster x item 상호작용 matrix로 계산 및 변환하는 파일입니다!!

In [1]:
import pandas as pd
import numpy as np
import openpyxl

from collections import Counter
from itertools import permutations
from tqdm import tqdm

In [2]:
data_path = '/opt/ml/input/data/raw_codishop/view/item/item_codi_id.xlsx'
preprocessed_path = '/opt/ml/input/data/asset_codishop/view/item/item.csv'
# preprocessed_path = './temp_color_midclass.xlsx'

item_df = pd.read_excel(data_path, engine='openpyxl')
preprocessed_df = pd.read_csv(preprocessed_path)

In [18]:
print (f"악세서리를 제외한 전체 Item의 수 : {len(preprocessed_df)}")
print (f"고유한 중분류의 수 : {len(preprocessed_df['mid_class'].unique())}")
print (f"클러스터 ID의 개수 : {len(preprocessed_df['cluster_id'].unique())}")
print (f"고유한 색상 ID의 수 : {len(preprocessed_df['color_id'].unique())}")

악세서리를 제외한 전체 Item의 수 : 1045
고유한 중분류의 수 : 56
클러스터 ID의 개수 : 655
고유한 색상 ID의 수 : 184


In [3]:
drop_indexes = list()
for i in range(len(item_df)):
    if item_df.iloc[i]["id"] not in preprocessed_df["id"].unique() :
        drop_indexes.append(i)

item_df = item_df.drop(drop_indexes)

item x item 상호작용 행렬을 생성하는 파일

In [4]:
items_in_codi = item_df.groupby('codi_id')['id'].apply(list)

In [5]:
# 코디 내에서 가능한 아이템들의 조합을 계산하는 코드
possible_combinations = list()
items_in_codi = item_df.groupby('codi_id')['id'].apply(list)

for val in tqdm(items_in_codi):
    val = sorted(val)
    comb_res = list(permutations(val, 2))
    possible_combinations.extend(comb_res)

100%|██████████| 334/334 [00:00<00:00, 260244.76it/s]


In [6]:
pivot_list = list()
for item1, item2 in possible_combinations:
    cluster_id = preprocessed_df[preprocessed_df['id'] == item1]['cluster_id'].values
    pivot_list.append((cluster_id[0], item2))

In [7]:
count_combination = list(Counter(pivot_list).most_common())

In [8]:
result = list()
for (cluster_id, item_id), cnt in count_combination:
    result.append((cluster_id, item_id, cnt))

In [9]:
IIM = pd.DataFrame(result, columns=["id", "item", "count"]).pivot_table(index="id", columns="item", values="count").fillna(0)
IIM = IIM.astype(int)
# TODO : 데이터 경로 설정
IIM.to_csv("/opt/ml/input/data/asset_codishop/view/item/itemInteractionMatrix_withColor.csv", index=True)

In [10]:
IIM

item,102621,104238,104239,104241,127121,212882,259538,260755,267464,276256,...,2449160,2467243,2482720,2482738,2492971,2495507,2514819,2515721,2527886,2540019
id,,,,,,,,,,,,,,,,,,,,,
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9530,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9586,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9764,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
ones = 0
zeros = 0
for i in range(len(IIM)) :
    oz_cnt = IIM.iloc[i].value_counts()
    zeros += oz_cnt[0]
    ones += sum(oz_cnt[1:])
print(f"Sparsity of this Data is : {zeros/(ones+zeros):.4}")

Sparsity of this Data is : 0.993


In [12]:
total_sum, total_cnt = 0, 0
max_val = 0
for idx, row in IIM.iterrows():
    max_val = max(max_val, max(row))
    total_sum += sum(row)
    total_cnt += len(row)
    # break

print (max_val)
print (total_sum, total_cnt, total_sum / total_cnt)

4
4896 683820 0.007159778889181363


In [13]:
sum(IIM) / (len(IIM.columns) * len(IIM.index))

2390.046833962154